In [ ]:
<VSCode.Cell language="markdown">
**Define Ordered Task List**
- We'll define simple tasks that log start/end and return results.
</VSCode.Cell>
<VSCode.Cell language="python">
import time, random, logging
logging.basicConfig(level=logging.INFO, format="%(message)s")
def task_factory(name, duration=0.1, fail_rate=0.0):
    def _task():
        logging.info(f"START {name}")
        time.sleep(duration)
        if random.random() < fail_rate:
            raise RuntimeError(f"{name} failed")
        res = {"task": name, "ts": time.time()}
        logging.info(f"END {name}")
        return res
    return _task
tasks = [task_factory(f"task_{i}", duration=0.05) for i in range(5)]
</VSCode.Cell>
<VSCode.Cell language="markdown">
**Execute Tasks Sequentially**
- `run_in_order(tasks)` executes tasks strictly in order, collecting results.
</VSCode.Cell>
<VSCode.Cell language="python">
def run_in_order(tasks, max_retries=0):
    results = []
    logs = []
    for idx, t in enumerate(tasks):
        attempt = 0
        while True:
            try:
                logs.append((idx, f"start:{idx}"))
                r = t()
                results.append((idx, r))
                logs.append((idx, f"end:{idx}"))
                break
            except Exception as e:
                attempt += 1
                logs.append((idx, f"error:{idx}:{e}"))
                if attempt > max_retries:
                    raise
                time.sleep(0.05)
    return results, logs
seq_results, seq_logs = run_in_order(tasks)
seq_results, seq_logs[:4]
</VSCode.Cell>
<VSCode.Cell language="markdown">
**Track Results and Order**
- Verify output indices match input order and logs are ordered.
</VSCode.Cell>
<VSCode.Cell language="python">
def verify_order(results, logs, n):
    idxs = [i for i, _ in results]
    assert idxs == list(range(n)), f"Order mismatch: {idxs}"
    # ensure logs are grouped by task index sequence
    ordered = True
    last_idx = -1
    for i, _ in logs:
        if i < last_idx:
            ordered = False
            break
        last_idx = i
    return ordered
verify_order(seq_results, seq_logs, len(tasks))
</VSCode.Cell>
<VSCode.Cell language="markdown">
**Handle Failures and Retries**
- Add retry policy (max_retries=2) and simulate failure.
</VSCode.Cell>
<VSCode.Cell language="python">
fail_tasks = [task_factory("ok1"), task_factory("flaky", fail_rate=0.7), task_factory("ok2")]
try:
    res, logs = run_in_order(fail_tasks, max_retries=2)
    print("Succeeded with retries", res)
except Exception as e:
    print("Still failed:", e)
</VSCode.Cell>
<VSCode.Cell language="markdown">
**Unit Tests for Ordered Execution**
- Use unittest to validate strict ordering and retry behavior.
</VSCode.Cell>
<VSCode.Cell language="python">
import unittest
class TestOrderedExecution(unittest.TestCase):
    def test_order(self):
        tasks_local = [task_factory(f"t{i}", duration=0.01) for i in range(3)]
        res, logs = run_in_order(tasks_local)
        self.assertTrue(verify_order(res, logs, len(tasks_local)))
    def test_retry(self):
        tasks_local = [task_factory("flaky", fail_rate=0.9), task_factory("t2")]
        res, logs = run_in_order(tasks_local, max_retries=2)
        # Either succeed or raise; if succeeded, ensure order
        if res:
            self.assertTrue(verify_order(res, logs, len(tasks_local)))
suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestOrderedExecution)
unittest.TextTestRunner(verbosity=2).run(suite)
</VSCode.Cell>
<VSCode.Cell language="markdown">
**Compare Ordered vs Parallel Execution**
- Demonstrate parallel execution completion order can differ.
</VSCode.Cell>
<VSCode.Cell language="python">
from concurrent.futures import ThreadPoolExecutor, as_completed
par_tasks = [task_factory(f"p{i}", duration=random.uniform(0.01, 0.08)) for i in range(6)]
with ThreadPoolExecutor(max_workers=3) as ex:
    futs = [ex.submit(t) for t in par_tasks]
    done = []
    for f in as_completed(futs):
        done.append(f.result())
print("Parallel completion order (by name):", [d["task"] for d in done])
</VSCode.Cell>